In [2]:
from datetime import datetime, timedelta, date
from typing import Annotated, Union

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from passlib.context import CryptContext
from pydantic import BaseModel, EmailStr
from enum import Enum
import psycopg2
import pandas as pd

In [3]:
DB_CON = {
        "host"      : 'rds-ticket-app-prod.czgosi3nm3l7.eu-central-1.rds.amazonaws.com',
        "database"  : 'public',
        "user"      : 'ticketappuser',
        "password"  : 'GA8yDbApaTthvqgH16V0jg==',
        'port'      : '5432'
    }

def format_value_for_sql(value):
    if value is None:
        return "NULL"
    elif isinstance(value, int):
        return str(value)
    elif isinstance(value, float):
        return str(value)
    elif isinstance(value, bool):
        return str(int(value))  # Convert bool to 0 or 1
    else:
        # If the data type is not recognized, raise an error or handle it accordingly
        return f"""'{str(value).replace("'", "''")}'"""

def read_data(sql):
    conn = psycopg2.connect(**DB_CON)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [str(col[0]) for col in cur.description ]
    conn.close()
    return pd.DataFrame(rows, columns=cols)

In [4]:
class Role(str, Enum):
    admin = "admin"
    user = "user"

class User(BaseModel):
    username: str
    email: EmailStr
    role: Role
    name: Union[str, None] = None
    disabled: Union[bool, None] = None

In [5]:
role = Role('admin')
user = User(username='med', email='med@rebike.com', role=role, name='medd', disabled=False)
user

User(username='med', email='med@rebike.com', role=<Role.admin: 'admin'>, name='medd', disabled=False)

In [ ]:
import psycopg2

try:
    # Establish a connection to the database
    conn = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()

    # Your SQL INSERT query with RETURNING clause
    insert_query = "INSERT INTO your_table (column1, column2) VALUES (%s, %s) RETURNING column1, column2, other_columns;"

    # Specify the values for the new record
    values = ('value1', 'value2')

    # Execute the INSERT query
    cursor.execute(insert_query, values)

    # Fetch the newly inserted row
    new_row = cursor.fetchone()

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    # Now, new_row contains the values of the columns specified in the RETURNING clause
    print("Newly inserted row:", new_row)

except psycopg2.Error as e:
    # Handle the exception (print or log the error message, rollback changes, etc.)
    print("Error:", e)

finally:
    # Ensure that the connection is properly closed
    if conn is not None:
        conn.close()


In [30]:

cur.executemany(sql, values)
conn.commit()
obj = [{
        "username": 'usdsder11',
        "email": 'user@email.com',
        "role": 'user',
        "name": 'new_user.name',
        "password": 'pwd',
        "disabled": False
        },
        {
        "username": 'usdsdedddwr11',
        "email": 'user@email.com',
        "role": 'user',
        "name": 'new_user.name',
        "password": 'pwd',
        "disabled": False
        }]
keys = ', '.join([f'"{o}"' for o in obj[0].keys()])
tags = ', '.join(['%s' for i in obj[0].values()])


b'\nInsert into users ("username", "email", "role", "name", "password", "disabled") values (%s, %s, %s, %s, %s, %s)\nreturning *\n'

In [37]:
conn = psycopg2.connect(**DB_CON)
cur = conn.cursor()
def insert_many(table: str, id_column: str, values: list):
    if not values:
        return []

    keys = values[0].keys()
    query = cur.mogrify("INSERT INTO {} ({}) VALUES {} RETURNING {}".format(
            table,
            ', '.join(keys),
            ', '.join(['%s'] * len(values)),
            id_column
        ), [tuple(v.values()) for v in values])
    cur.execute(query)
    cols = [str(col[0]) for col in cur.description ]
    values = cur.fetchall()
    return  [dict(zip(cols,val)) for val in values]

In [38]:
insert_many('users', '*', obj)

[{'id': 23,
  'username': 'usdsder11',
  'email': 'user@email.com',
  'password': 'pwd',
  'role': 'user',
  'name': 'new_user.name',
  'disabled': False},
 {'id': 24,
  'username': 'usdsdedddwr11',
  'email': 'user@email.com',
  'password': 'pwd',
  'role': 'user',
  'name': 'new_user.name',
  'disabled': False}]

In [28]:
vals = cur.fetchone()

ProgrammingError: no results to fetch

{'id': 15,
 'username': 'usdsder11',
 'email': 'user@email.com',
 'password': 'pwd',
 'role': 'user',
 'name': 'new_user.name',
 'disabled': False}

In [57]:
def update_one(id, id_col, obj, table):
    try:
        conn = psycopg2.connect(**DB_CON)
        cur = conn.cursor()
        obj = {
                "username": 'hacked',
                "email": 'hacked@email.com',
                "role": 'user',
                "name": 'hacked',
                "password": 'hacked',
                "disabled": True
                }
        returns = ', '.join([f'"{o}"' for o in obj.keys()])
        to_update = ',\n'.join([f'''"{k}"=%s''' for k in obj])
        values = list(obj.values()) + [id]
        sql = f"""
        UPDATE {table} SET
        {to_update}
        where "{id_col}" = %s
        returning {returns}
        """
        print(sql)
        cur.execute(sql, values)
        cols = [str(col[0]) for col in cur.description ]
        vals = cur.fetchone()
        conn.commit()
        cur.close()
        conn.close()
        return dict(zip(cols, vals))
    except (psycopg2.Error) as error:
        print(error)
        cur.close()
        conn.close()
        return False

In [58]:
obj = {
        "username": 'hacked',
        "email": 'hacked@email.com',
        # "role": 'user',
        # "name": 'hacked',
        # "password": 'hacked',
        # "disabled": True
        }
a = update_one(17, 'id', obj, 'users')
print(a)


        UPDATE users SET
        "username"=%s,
"email"=%s,
"role"=%s,
"name"=%s,
"password"=%s,
"disabled"=%s
        where "id" = %s
        returning "username", "email", "role", "name", "password", "disabled"
        
{'username': 'hacked', 'email': 'hacked@email.com', 'role': 'user', 'name': 'hacked', 'password': 'hacked', 'disabled': True}


In [45]:


table = 'users'
id_col = 'processId'
returns = ', '.join([f'"{o}"' for o in obj.keys()])
to_update = ',\n'.join([f'''"{k}"=%s''' for k in obj])

sql = f"""
UPDATE {table} SET
{to_update}
where "{id_col}" = %s
returning {returns}
"""
print(sql)


UPDATE users SET
"username"=%s,
"email"=%s,
"role"=%s,
"name"=%s,
"password"=%s,
"disabled"=%s
where "processId" = %s
returning "username", "email", "role", "name", "password", "disabled"



In [64]:
from typing import Annotated, Union, List
class User(BaseModel):
    username: str
    email: EmailStr
    role: str # Role
    name: Union[str, None] = None
    disabled: Union[bool, None] = None

class Gender(str, Enum):
    Mr = "Mr"
    Mrs = "Mrs"
    Other = "Other"


class Status(str, Enum):
    Open = 'Open'
    Paid = 'Paid'
    Invoiced = 'Invoiced'
    Completed = 'Completed'
    Waiting = 'Waiting'
    ReadyToInvoice = 'ReadyToInvoice'
    InboundCheck = 'InboundCheck'
    Received = 'Received'
    PickedUp = 'PickedUp'
    InPickUp = 'InPickUp'
    Avis = 'Avis'
    Canceled = 'Canceled'
    Reapply = 'Reapply'
    PickUpFailed = 'PickUpFailed'


class PickUpLogistics(str, Enum):
    Wiechert = 'Wiechert'
    Emons = 'Emons'
    Gaehler = 'Gaehler'
    Alltrans = 'Alltrans'

class Bike(str, Enum):
    Bicycle = 'Bicycle'
    Pedelec = 'Pedelec'
    Other = 'Other'

class Color(str, Enum):
    Yellow = 'Yellow'
    Gold = 'Gold'
    Organe = 'Organe'
    Red = 'Red'
    Pink = 'Pink'
    Purple = 'Purple'
    Blue = 'Blue'
    Turquoise = 'Turquoise'
    Green = 'Green'
    Grey = 'Grey'
    Brown = 'Brown'
    Copper = 'Copper'
    White = 'White'
    Black = 'Black'
    Silver = 'Silver'
    Multicolor = 'Multicolor'

class communicationError(str, Enum):
    Bounced = 'Bounced'
    IncorrectPhone = 'IncorrectPhone'
    none = 'None'

class userRole(str, Enum):
    BackendUserLeasing = 'BackendUserLeasing'
    FrontUser = 'FrontUser'
    SuperBackendUser = 'SuperBackendUser'
    BackendUserCustomerCare = 'BackendUserCustomerCare'
    BackendUserLogisticsWI = 'BackendUserLogisticsWI'
    BackendUserLogisticsEM = 'BackendUserLogisticsEM'
    BackendUserLogisticsGA = 'BackendUserLogisticsGA'
    BackendUserLogisticsAT = 'BackendUserLogisticsAT'

class FormsToUpdate(BaseModel):
    # mandatory
    processIds: List[int]
    userFirstName: Union[str, None] = None
    userLastName: Union[str, None] = None 
    userEmail: Union[EmailStr, None] = None 
    userTelephone: Union[str, None] = None 
    userStreet: Union[str, None] = None 
    userHouseNumber: Union[str, None] = None 
    userZipCode: Union[str, None] = None 
    userCity: Union[str, None] = None 
    userCountry: Union[str, None] = None 
    bikeBrand: Union[str, None] = None 
    bikeModel: Union[str, None] = None 
    bikeRrp: Union[float, None] = None 
    bikeAccessories: Union[str, None] = None 
    bikeFrameNumber: Union[str, None] = None 
    contractStartDate: Union[date, None] = None 
    contractAmount: Union[float, None] = None 
    # optional
    pickDate: Union[date, None] = None
    bikeSize: Union[str, None] = None
    bikeId: Union[int, None] = None
    contractSettlementPrice: Union[float, None] = None
    contractSettlementDate: Union[date, None] = None
    status: Union[Status, None] = None
    pickUpLogistics: Union[PickUpLogistics, None] = None
    statusComment: Union[str, None] = None
    contractNumber: Union[str, None] = None
    pickUpFeedback: Union[str, None] = None
    bikeAdditionalText: Union[str, None] = None
    bikeBatteryNumber: Union[str, None] = None
    bikeColor: Union[Color, None] = None
    bikeKind: Union[str, None] = None
    bikeModelYear: Union[int, None] = None
    bikeOriginalPurchasePrice: Union[float, None] = None
    bikeOriginalPurchasePriceNet: Union[float, None] = None
    bikeUser: Union[str, None] = None
    contractAttachment: Union[str, None] = None
    contractCommunicationError: Union[communicationError, None] = None
    contractConditionBasedReduction: Union[float, None] = None
    contractDeliveryInformation: Union[str, None] = None
    contractPartsBasedReduction: Union[float, None] = None
    contractPartsMissing: Union[str, None] = None
    contractSentToLogistics: Union[bool, None] = None
    contractStateChangedDate: Union[date, None] = None
    contractSubmitDate: Union[date, None] = None
    pickContractReceived: Union[bool, None] = None
    pickEarliest: Union[date, None] = None
    pickTimeFrom: Union[str, None] = None
    pickTimeTill: Union[str, None] = None
    pickUpOkay: Union[bool, None] = None
    userCompany: Union[str, None] = None
    userDeviatingPickup: Union[bool, None] = None
    userDeviatingPickupAddress: Union[str, None] = None
    # userEmailId: Union[str, None] = None
    userFederalState: Union[str, None] = None
    userFunction: Union[str, None] = None
    userGender: Union[Gender, None] = None
    userMobilePhone: Union[str, None] = None
    bikeType: Union[Bike, None] = None

In [67]:
current_user = User(username='med', email='email@email.com', role='admin', name='med', disabled='True')
new_contract = FormsToUpdate(processIds=[1,2,3], userFirstName='med', pickUpLogistics= 'Emons')
new_contract 


FormsToUpdate(processIds=[1, 2, 3], userFirstName='med', userLastName=None, userEmail=None, userTelephone=None, userStreet=None, userHouseNumber=None, userZipCode=None, userCity=None, userCountry=None, bikeBrand=None, bikeModel=None, bikeRrp=None, bikeAccessories=None, bikeFrameNumber=None, contractStartDate=None, contractAmount=None, pickDate=None, bikeSize=None, bikeId=None, contractSettlementPrice=None, contractSettlementDate=None, status=None, pickUpLogistics=<PickUpLogistics.Emons: 'Emons'>, statusComment=None, contractNumber=None, pickUpFeedback=None, bikeAdditionalText=None, bikeBatteryNumber=None, bikeColor=None, bikeKind=None, bikeModelYear=None, bikeOriginalPurchasePrice=None, bikeOriginalPurchasePriceNet=None, bikeUser=None, contractAttachment=None, contractCommunicationError=None, contractConditionBasedReduction=None, contractDeliveryInformation=None, contractPartsBasedReduction=None, contractPartsMissing=None, contractSentToLogistics=None, contractStateChangedDate=None, co

In [76]:
keys_to_update = [ k for k,v in new_contract.__dict__.items() if v and k != 'processIds']
enums = ['status', 'pickUpLogistics', 'bikeColor', 'contractCommunicationError', 'userGender', 'bikeType']
obj = {
    k: new_contract.__dict__[k] if k not in enums else new_contract.__dict__[k].value
    for k in keys_to_update
}
obj

{'userFirstName': 'med', 'pickUpLogistics': 'Emons'}

In [75]:
current_user.__dict__[k]

C:\Users\mohamed benzarti\AppData\Local\Temp\ipykernel_11076\3213987694.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  current_user.schema()


{'properties': {'username': {'title': 'Username', 'type': 'string'},
  'email': {'format': 'email', 'title': 'Email', 'type': 'string'},
  'role': {'title': 'Role', 'type': 'string'},
  'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Name'},
  'disabled': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
   'default': None,
   'title': 'Disabled'}},
 'required': ['username', 'email', 'role'],
 'title': 'User',
 'type': 'object'}

In [ ]:
obj = {
        'submittingUser' : current_user.username,
        'userFirstName' : new_contract.userFirstName,
        'userLastName' : new_contract.userLastName,
        'userEmail' : new_contract.userEmail,
        'userTelephone' : new_contract.userTelephone,
        'userStreet' : new_contract.userStreet,
        'userHouseNumber' : new_contract.userHouseNumber,
        'userZipCode' : new_contract.userZipCode,
        'userCity' : new_contract.userCity,
        'userCountry' : new_contract.userCountry,
        'bikeBrand' : new_contract.bikeBrand,
        'bikeModel' : new_contract.bikeModel,
        'bikeRrp' : new_contract.bikeRrp,
        'bikeAccessories' : new_contract.bikeAccessories,
        'bikeFrameNumber' : new_contract.bikeFrameNumber,
        'contractStartDate' : new_contract.contractStartDate,
        'contractAmount' : new_contract.contractAmount,
        'pickDate' : new_contract.pickDate,
        'bikeSize' : new_contract.bikeSize,
        'bikeId' : new_contract.bikeId,
        'contractSettlementPrice' : new_contract.contractSettlementPrice,
        'contractSettlementDate' : new_contract.contractSettlementDate,
        'status' : new_contract.status.value,
        'pickUpLogistics' : new_contract.pickUpLogistics.value,
        'statusComment' : new_contract.statusComment,
        'contractNumber' : new_contract.contractNumber,
        'pickUpFeedback' : new_contract.pickUpFeedback,
        'bikeAdditionalText' : new_contract.bikeAdditionalText,
        'bikeBatteryNumber' : new_contract.bikeBatteryNumber,
        'bikeColor' : new_contract.bikeColor.value,
        'bikeKind' : new_contract.bikeKind,
        'bikeModelYear' : new_contract.bikeModelYear,
        'bikeOriginalPurchasePrice' : new_contract.bikeOriginalPurchasePrice,
        'bikeOriginalPurchasePriceNet' : new_contract.bikeOriginalPurchasePriceNet,
        'bikeUser' : new_contract.bikeUser,
        'contractAttachment' : new_contract.contractAttachment,
        'contractCommunicationError' : new_contract.contractCommunicationError.value,
        'contractConditionBasedReduction' : new_contract.contractConditionBasedReduction,
        'contractDeliveryInformation' : new_contract.contractDeliveryInformation,
        'contractPartsBasedReduction' : new_contract.contractPartsBasedReduction,
        'contractPartsMissing' : new_contract.contractPartsMissing,
        'contractSentToLogistics' : new_contract.contractSentToLogistics,
        'contractStateChangedDate' : new_contract.contractStateChangedDate,
        'contractSubmitDate' : new_contract.contractSubmitDate,
        'pickContractReceived' : new_contract.pickContractReceived,
        'pickEarliest' : new_contract.pickEarliest,
        'pickTimeFrom' : new_contract.pickTimeFrom,
        'pickTimeTill' : new_contract.pickTimeTill,
        'pickUpOkay' : new_contract.pickUpOkay,
        'userCompany' : new_contract.userCompany,
        'userDeviatingPickup' : new_contract.userDeviatingPickup,
        'userDeviatingPickupAddress' : new_contract.userDeviatingPickupAddress,
        'userFederalState' : new_contract.userFederalState,
        'userFunction' : new_contract.userFunction,
        'userGender' : new_contract.userGender.value,
        'userMobilePhone' : new_contract.userMobilePhone,
        'bikeType' : new_contract.bikeType.value
    }